In [1]:
from selenium import webdriver 
import time
from tqdm.notebook import tqdm
import pandas as pd
import json, csv

import warnings
warnings.filterwarnings('ignore')

In [2]:
files = ['bproperty_bashundhara_RA.csv',
         'bproperty_dhanmondi.csv',
         'bproperty_gulshan.csv',
         'bproperty_mirpur.csv',
         'bproperty_uttara.csv' ]

dfs = []
for file in files:
    dfs.append(pd.read_csv(file))
df = pd.concat(dfs, ignore_index=True)

In [3]:
def get_unique_id(url):
    url_ = url
    unique = str(url_).replace('/en/property/details-', '').replace('.html', '')
    return unique

def fix_url(url):
    url = 'https://www.bproperty.com' + str(url)
    return url

df['unique_id'] = df['url'].apply(get_unique_id)
df['url'] = df['url'].apply(fix_url)

In [4]:
# df.to_csv('combined.csv', index=False)
df.head()

,dimensions,num_beds,num_baths,house_type,city,location,area_block,description,url,price,unique_id
0,"2,055 sqft",4,4,Apartment,Dhaka,Bashundhara R-A,Block D,2055 Sq Ft Cozy Flat For Rent In Bashundhara R...,https://www.bproperty.com/en/property/details-...,35000.0,5237946
1,"1,911 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,Emanate Your Knack For Gardening By Renting Th...,https://www.bproperty.com/en/property/details-...,45000.0,3875341
2,"1,200 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block K,"To Secure Your Better State Of Living, Conside...",https://www.bproperty.com/en/property/details-...,20000.0,5182996
3,"1,910 sqft",3,3,Apartment,Dhaka,Bashundhara R-A,Block A,For Rental Purpose 1910 Sq Ft Commendable Desi...,https://www.bproperty.com/en/property/details-...,45000.0,3875358
4,800 sqft,2,2,Apartment,Dhaka,Bashundhara R-A,Block G,Bright And Cozy Apartment Featuring 800 Sq Ft ...,https://www.bproperty.com/en/property/details-...,18000.0,3651253


In [5]:
driver = webdriver.Firefox(executable_path='driver/geckodriver.exe')
tertiary_scrape = {}
bad_id = []

for i in tqdm(range(len(df))):
    try:
        url = df['url'][i]
        title = driver.get(url)

        # Click on JS Map
        driver.find_element_by_xpath('//div[@class="_4866b17e"]').click()
        time.sleep(5)

        # Click on required tabs and retrieve data
        schools = []
        restaurants = []
        hospitals = []
        parks = []

        tabs = driver.find_elements_by_xpath('//li[@class="a17d0d0d"]')

        for j in range(4):
            tabs[j].click()
            time.sleep(5)

            nearby = driver.find_elements(by='xpath', value='//div[@class="daf61ad1"]')

            if j == 0:
                for item in nearby:
                    schools.append(item.text)

            if j == 1:
                for item in nearby:
                    restaurants.append(item.text)

            if j == 2:
                for item in nearby:
                    hospitals.append(item.text)

            if j == 3:
                for item in nearby:
                    parks.append(item.text) 

        tertiary_scrape[ df['unique_id'][i] ] = {'schools': len(list(set(schools))),
                                                 'restaurants': len(list(set(restaurants))),
                                                 'hospitals': len(list(set(hospitals))),
                                                 'parks': len(list(set(parks)))}
        
        fields = [i, df['unique_id'][i], len(list(set(schools))), len(list(set(restaurants))),
                                      len(list(set(hospitals))), len(list(set(parks)))]
        
        with open(r'tertiary.csv', 'a') as f:
            writer = csv.writer(f)
            writer.writerow(fields)
            f.close()
            
    except Exception as e:
        print('ID: ' + str(df['unique_id'][i]))
        bad_id.append(df['unique_id'][i])
        
        print("Retrying in: 10 mins")
        for s in tqdm(range(600)):
            time.sleep(1)
        
        driver.quit()
        print('Starting New Instance')
        print()
        
        driver = webdriver.Firefox(executable_path='driver/geckodriver.exe')
        

driver.quit()

  0%|          | 0/13415 [00:00<?, ?it/s]

ID: 5237946
Retrying in: 10 mins


  0%|          | 0/600 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
tertiary_scrape

In [ ]:
scraped = pd.DataFrame(tertiary_scrape)
scraped = scraped.T.reset_index()
scraped.rename(columns={'index':'unique_id'}, inplace=True)

scraped

In [ ]:
bad_id